**Avant de débuter ce TP** :

1. **Changez le type d'exécution sur Google Colab** : `Exécution > Modifiez le type d'exécution > T4 GPU`
2. **Installez les paquets ci-dessous** :

In [ ]:
! pip install lightning torchmetrics torchinfo pyconll

3. Exécutez ce code pour supprimer quelques messages et avertissements éventuellement affichés.

In [2]:
import logging
logging.getLogger("lightning").setLevel(logging.ERROR)
logging.getLogger("lightning.pytorch.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("lightning.pytorch.accelerators.cuda").setLevel(logging.WARNING)
logger = logging.getLogger("lightning")
logger.propagate = False

import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")
warnings.filterwarnings("ignore", ".*Missing logger folder.*")

# Exercice de grammaire par réseau de neurones récurrent

Dans ce notebook, nous allons travailler sur le jeu de données [noun verb](https://github.com/google-research-datasets/noun-verb).
Ce jeu de données contient des phrases en anglais naturelles où il peut y avoir une certaine ambiguïté sur le rôle d'un mot, c'est-à-dire de savoir s'il s'agit d'un nom ou d'un verbe.

Vous trouverez ci-dessous quelques observations de ce jeu de données illustrant la tâche. Le mot en gras est le mot à classer et le mot en italique après la phrase indique la bonne réponse :

> Certain insects can damage plumerias, such as mites, **flies**, or aphids. *NOUN*

> **Mark** which area you want to distress. *VERB*

Par exemple, le mot **flies** peut correspondre à la troisième personne au singulier du verbe *fly* (par exemple *time flies by so fast*), mais également au pluriel du nom *fly* (mouche).
De même, le mot **mark** peut correspondre au verbe *mark* (marquer) ou au nom *mark* (marque).

Le tâche est donc une classification binaire car il n'y a que deux classes possibles : *nom* ou *verbe*.

## (Télé)chargement du jeu de données

Le jeu de données est déjà séparé en trois jeux d'entraînement, de validation et d'évaluation.
Les fichiers correspondants (`train.conll`, `dev.conll` and `test.conll`) sont disponibles sur ce [dépôt GitHub](https://github.com/google-research-datasets/noun-verb).
La fonction `load_dataset` permet de charger (et télécharger si besoin) les trois jeux de données.

In [3]:
def load_dataset(path='data'):
    """Load the noun verb dataset.

    Parameters
    ----------
    path : str
        Chemin du répertoire.

    Returns
    -------
    train : Conll
        Jeu d'entraînement.

    validation : Conll
        Jeu de validation.

    test : Conll
        Jeu d'évaluation.

    """
    import os
    import pyconll
    from urllib.request import urlretrieve

    if not os.path.exists(path):
        os.makedirs(path)

    files = ('train.conll', 'dev.conll', 'test.conll')

    # Downloads the files if necessary
    for file in files:
        if not os.path.isfile(os.path.join(path, file)):
            url = f'https://raw.githubusercontent.com/google-research-datasets/noun-verb/master/{file}'
            urlretrieve(url, os.path.join(path, file))

    return (pyconll.load_from_file(os.path.join(path, file)) for file in files)

Il suffit d'exécuter la fonction pour (télé)charger les trois jeux de données :

In [4]:
train, validation, test = load_dataset()

Visualisons la tâche d'apprentissage automatique.
La fonction `print_sample()` affiche l'observation d'un jeu de données correspondant à l'indice fourni.

In [5]:
def print_sample(dataset, idx):
    """Affiche une observation d'un jeu de données.

    Parameters
    ----------
    dataset : Conll
        Dataset.

    idx : int
        Indice de l'observation à afficher.

    """
    import re

    if not 0 <= idx < len(dataset):
        raise ValueError("Invalid index.")

    label = None
    index = None

    for i, token in enumerate(dataset[idx]):
        if len(token.feats):
            if label is not None:
                raise ValueError("Invalid sample. Try another index.")
            label = next(iter(token.feats['POS']))
            index = i

    if label is None:
        raise ValueError("Invalid sample. Try another index.")

    sentence = ' '.join([token.form for token in dataset[idx]])

    # Punctuation formatting
    for char in ",:;.?!)'":
        pattern = f' *\\{char}' if char in '.?)' else f' *{char}'
        sentence = re.sub(pattern, char, sentence)

    for char in "(`":
        pattern = f'\\{char} *' if char in '(' else f'{char} *'
        sentence = re.sub(pattern, char, sentence)

    res = 'Sentence' + '\n' + '=' * 8 + '\n'
    res += sentence
    res += '\n\n' + 'Word' + '\n' + "=" * 4 + '\n'
    res += dataset[idx][index].form + f' (at position {index})'
    res += '\n\n' + 'Label' + '\n' + "=" * 5 + '\n'
    res += label

    print(res)

### Question 1

Exécutez cette fonction pour afficher quelques observations des jeux d'entraînement, de validation et d'évaluation. 

In [6]:
print_sample(train, 0)

Sentence
License plates of cars from your area or your destination.

Word
====
License (at position 0)

Label
=====
NON-VERB


In [7]:
print_sample(validation, 0)

Sentence
Once you have the new glass, cleaning and replacing it is all that's left.

Word
====
cleaning (at position 7)

Label
=====
VERB


In [8]:
print_sample(test, 0)

Sentence
Retail goods were seldom sold with shipping any faster than overnight.

Word
====
shipping (at position 6)

Label
=====
NON-VERB


## Représentations vectorielles de mots

La tâche à effectuer relève du **traitement automatique des langues** car les données en entrée sont des phrases (en anglais).
Une phrase est une séquence de **tokens**.
Un token est une unité d'une langue.
Par exemple, les mots sont des tokens, mais les signes de ponctuation (virgule, point d'exclamation, etc.) sont également des tokens.
Cependant, il n'est pas pratique de travailler directement avec des tokens : les modèles d'apprentissage automatique sont des fonctions mathématiques dont les entrées sont *numériques*.
Il est néamoins possible de **transformer les tokens en vecteurs numériques**.
De telles représentations sont connues en anglais sous le terme *embeddings* et nous utiliserons le terme **représentations vectorielles** en français.

Une approche triviale serait de transformer chaque mot en un vecteur ne contenant que des zéros à part pour un élément dont la valeur serait 1.
Cette approche est connue en apprentissage automatique sous le terme **encodage *one-hot*** et peut être utilisée pour transformer une variable catégorielle en plusieurs variables binaires.
Cependant, cette approche a de nombreux inconvénients :

* Il y a de nombreux mots différents dans une langue, par exemple des centaines de milliers en anglais et en français.
* Les données ont été transformées, mais aucune information pertinente n'a été extraite dans cete représentation.

Une approche plus utile est de **transformer un mot en un vecteur numérique dense de petite dimension**.
Ici, nous n'allons pas apprendre nous-mêmes ces représentations, mais nous allons utiliser des représentations disponibles publiquement qui ont été apprises sur des jeux de données très grands.

Une collection de telles représentations est connue sous le nom de [GloVe](https://nlp.stanford.edu/projects/glove/) (pour *Global Vectors for Word Representation*).
Il existe plusieurs versions de ces représentations qui dépendent des jeux de données sur lesquels ces modèles ont été entraînés et de la taille des représentations.
Nous allons utiliser la première version de GloVe, qui a été entraîné sur Wikipedia et sur des articles de presse.
Il existe quatre versions pour les représentations avec des dimensions différentes : $50$, $100$, $200$ et $300$.

La commande ci-dessous permet de télécharger l'archive contenant les représentations dans les quatre différentes tailles de dimension.

In [9]:
! cd data && wget https://nlp.stanford.edu/data/glove.6B.zip && unzip glove.6B.zip

--2025-05-02 19:24:03--  https://nlp.stanford.edu/data/glove.6B.zip
Résolution de nlp.stanford.edu (nlp.stanford.edu)… 171.64.67.140
Connexion à nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443… connecté.
requête HTTP transmise, en attente de la réponse… 301 Moved Permanently
Emplacement : https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [suivant]
--2025-05-02 19:24:04--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Résolution de downloads.cs.stanford.edu (downloads.cs.stanford.edu)… 171.64.64.22
Connexion à downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 862182613 (822M) [application/zip]
Sauvegarde en : « glove.6B.zip »

glove.6B.zip         80%[===============>    ] 664,56M  --.-KB/s    ds 30m 55s 

2025-05-02 19:54:59 (367 KB/s) — Erreur de lecture à l’octet 696844024/862182613 (Operation timed out). Nouvel essai.

--2025-05-02 19:55:00--  (essai :  2)  https

Nous allons utiliser les représentations les plus petites, c'est-à-dire en dimension $50$. **En résumé, nous allons transformer chaque token en un vecteur numérique dense de dimension 50**.

Pour les tokens qui ne font pas partie du vocabulaire du jeu d'entraînement, on va créer un token spécifique, appelé `out_of_vocabulary`, que l'on va calculer comme le vecteur moyen de toutes les représentations.

La fonction `load_embeddings()` charge les données dans un dictionnaire et calcule la représentation moyenne pour le token `out_of_vocabulary`.
Chaque clé du dictionnaire est un token et sa valeur est la représentation vectorielle de ce token. 

In [10]:
import numpy as np
import torch


def load_embeddings(path='data'):
    """Charge les représentations vectorielles GloVe 50d.

    Parameters
    ----------
    path : str
        Chemin du le répertoire.

    Returns
    -------
    embeddings : dict
        Représentations vectorielles GloVe 50d.

    """
    import os

    # Save the results in a dictionary
    embeddings = {}

    # Get the embedding for each token in the file
    mean = torch.zeros(50, dtype=torch.float32)
    with open(os.path.join(path, 'glove.6B.50d.txt'), 'r') as file:
        for line in file.readlines():
            split = line.strip().split(' ')
            token = split[0]
            value = torch.from_numpy(np.array(split[1:], dtype=np.float32))
            embeddings[token] = value
            mean += value

    # Define the mean embedding for out-of-vocabulary token
    embeddings['out_of_vocabulary'] = mean / len(embeddings)

    return embeddings

On exécute cette fonction pour (télé)charger ces représentations vectorielles de mots anglais.

In [11]:
embeddings = load_embeddings()

### Question 2

Affichez les représentations vectorielles de quelques mots anglais courants.

> **Remarque** : Tous les mots dans le dictionnaire sont en minuscules.

In [12]:
embeddings['the']

tensor([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
        -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
         2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
         1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
        -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
        -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
         4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
         7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
        -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
         1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01])

In [13]:
embeddings['chess']

tensor([-1.2300,  0.5334, -0.5474,  0.7492,  0.0514, -0.0492,  0.3764,  0.1278,
        -1.2388, -0.1074,  0.6666, -0.0176, -0.7533,  1.1521,  0.7994, -0.7221,
        -0.0103,  1.1645, -0.6130,  0.0896,  0.1537, -0.5346, -0.3841,  1.1418,
         0.0992, -0.9156, -0.8442, -1.0483, -1.0846, -0.1507,  1.2818, -0.3466,
        -0.1552, -0.2263, -0.6132,  0.7580, -0.2891,  1.0317, -0.3066, -0.7005,
         0.7052, -0.5768, -0.0880,  0.6581,  0.0929,  0.3469,  1.1974,  0.2684,
        -0.8472, -0.5815])

In [14]:
embeddings['hell']

tensor([ 0.2754,  0.4626,  0.1040, -1.1060,  0.3350, -0.2701,  0.2974,  0.2111,
         0.0397,  0.3216, -0.8549,  0.6873, -0.8040,  0.4050,  0.8508,  0.1374,
         0.9749,  0.3629, -0.5528,  0.1289, -0.2328,  0.0035,  0.4307,  0.6868,
         0.6574, -1.2946, -1.3315,  0.3312,  1.0792, -0.6563,  1.5733, -0.0137,
        -0.4854,  0.3714, -0.9064,  0.1826,  0.3905, -1.0533,  0.3809, -0.3286,
        -0.1469, -0.1961, -0.8044,  0.4517,  0.0356,  0.1807,  0.2160, -1.3607,
        -0.1665,  0.0376])

In [15]:
embeddings['storm']

tensor([ 7.6648e-01, -4.3500e-02,  1.0040e+00,  2.3024e-02, -1.3752e+00,
        -5.9183e-01, -4.4778e-01,  9.9807e-01,  4.8704e-01, -5.8070e-01,
        -1.3704e-01, -1.5639e-01,  4.3268e-01,  4.2912e-01, -2.6620e-02,
        -3.4538e-01, -6.2523e-01, -5.3198e-01, -1.4654e+00, -4.0254e-01,
        -7.6131e-01, -2.0400e-01,  3.0217e-01, -8.2802e-01,  7.8480e-01,
        -7.4680e-01, -1.2954e-01,  4.0479e-01,  6.9253e-01,  5.6607e-01,
         2.5380e+00,  4.3007e-02,  6.4473e-01, -7.3114e-01, -7.4961e-01,
         2.6799e-01, -1.2653e-01, -1.2082e+00,  4.8714e-01, -4.0238e-01,
        -1.0929e+00,  8.2982e-01,  4.1134e-01, -1.2558e+00, -1.3823e-01,
         5.6126e-01, -6.8973e-04, -1.4938e+00, -8.3548e-02, -7.3062e-02])

### Question 3

Les représentations vectorielles apprises sont connues pour avoir certaines structures linéaires.

* Calculez la représentation vectorielle `king + (woman - man)` et comparez-la à la représentation vectorielle de `queen`.
* Calculez la représentation vectorielle `strong + (lighter - light)` et comparez-la à la représentation vectorielle de `lighter`.

Vous pouvez utiliser la fonction *similarité cosinus* pour déterminer la similitude entre deux vecteurs. La similarité cosinus est définie par :
$$
    \cos(x, y) = \frac{x^\top y}{\Vert x \Vert \Vert y \Vert}
$$
Ses valeurs sont comprises dans l'intervalle $[-1, 1]$. Plus sa valeur est élevée, plus les vecteurs sont similaires.
La fonction `cosine_similarity()` définie ci-dessous calcule la similarité cosinus entre deux tenseurs.

In [16]:
def cosine_similarity(x, y):
    """Calcule la similarité cosinus entre deux tenseurs.

    Parameters
    ----------
    x, y : Tensor, shape = (embedding_size,)
        Les deux tenseurs à comparer.

    Returns
    -------
    float
        La similarité cosinus entre les deux tenseurs.
    """
    return ((x @ y) / ((x @ x) * (y @ y)) ** 0.5).item()

In [17]:
cosine_similarity(
    embeddings['king'] + embeddings['woman'] - embeddings['man'],
    embeddings['queen']
)

0.8609580993652344

In [18]:
cosine_similarity(
    embeddings['strong'] + embeddings['lighter'] - embeddings['light'],
    embeddings['stronger']
)

0.821943998336792

## Prétraitement des données

Maintenant qu'on a des représentations vectorielles pour les tokens et une meilleure compréhension des données, il est temps de prétraiter ces données.
Pour chaque observation, on va :
* transformer la séquence de tokens en une séquence de tenseurs,
* récupérer le label du token à classer, et
* récupérer l'indice du token à classer.

La fonction `preprocess_dataset()` effectue ce traitement sur le jeu de données fourni en entrée et renvoie ces trois variables.

In [19]:
def preprocess_dataset(dataset):
    """Prétraite un jeu de données.

    Parameters
    ----------
    dataset : Conll
        Jeu de données.

    Returns
    -------
    X : list[Tensors]
        Phrases prétraitées.

    y : Tensor
        Labels rétraités.

    index : Tensor
        Indices des tokens à prédire.
    """

    X = []
    y = []
    index = []

    label_mapping = {'NON-VERB': 0, 'VERB': 1}

    for sentence in dataset:

        embedded_sentence = []
        label = None
        idx = None

        for i, token in enumerate(sentence):

            # Get the token in lower cases
            token_lower = token.form.lower()

            # Get the embedding of the token
            if token_lower in embeddings.keys():
                embedded_sentence.append(embeddings[token_lower].reshape(1, -1))
            else:
                embedded_sentence.append(embeddings['out_of_vocabulary'].reshape(1, -1))

            # Get the label (if any)
            if len(token.feats):
                if label is not None:
                    raise ValueError("Two annotated tokens in a single sentence.")
                label = label_mapping[next(iter(token.feats['POS']))]
                idx = i

        # Add the preprocessed sample to the dataset only if there is a label available
        if label is not None:
            X.append(torch.concat(embedded_sentence))
            y.append(label)
            index.append(idx)

    y = torch.tensor(y).to(dtype=torch.float32)
    index = torch.tensor(index).to(dtype=torch.int64)

    return X, y, index

On n'a qu'à exécuter cette fonction sur les trois jeux de données.

In [20]:
X_train, y_train, index_train = preprocess_dataset(train)
X_val, y_val, index_val = preprocess_dataset(validation)
X_test, y_test, index_test = preprocess_dataset(test)

### Question 4

Les tenseurs `y_train`, `y_val` et `y_test` sont des tenseurs binaires (chaque élément vaut $0$ ou $1$). Un $0$ correspond à un nom, tandis qu'un $1$ correspond à un verbe.
Calculez la proportion de verbes sur les trois jeux pour avoir une idée de la précision (*accuracy*), c'est-à-dire la proportion de bonnes prédictions, d'un modèle trivial.
Vous pouvez utiliser la méthode [`torch.Tensor.mean()`](https://pytorch.org/docs/stable/generated/torch.Tensor.mean.html).

In [21]:
print(f"Propotion de verbes dans le jeu d'entraînement : {y_train.mean().item():.2%}")
print(f"Propotion de verbes dans le jeu de validation : {y_val.mean().item():.2%}")
print(f"Propotion de verbes dans le jeu d'évaluation : {y_test.mean().item():.2%}")

Propotion de verbes dans le jeu d'entraînement : 63.57%
Propotion de verbes dans le jeu de validation : 66.37%
Propotion de verbes dans le jeu d'évaluation : 65.82%


## Unité récurrente fermée bidirectionnelle 

### Unité récurrente fermée

L'**unité récurrente fermée** est un type de couche récurrente.
Il s'agit d'une variante simplifiée de la *longue mémoire de court terme* :

* Les **portes d'oubli et d'entrée** sont fusionnées en une seule **porte de mise à jour**.
* La **cellule de mémoire** et l'**état caché** sont fusionnés en un seul **état caché**.

L'image ci-dessous résume l'architecture de l'unité récurrente fermée :

[<img src="../figures/gru_full.png" width="500"/>](../figures/gru_full.png)

### Couche récurrente bidirectionnelle

On souhaite classer un mot spécifique dans une phrase donnée, soit en nom soit en verbe.
Pour effectuer cette tâche manuellement, un humain utiliserait la phrase en entier pour classer le mot.
Si on utilise toute la séquence de tokens dans une couche récurrente classique, le modèle ne sait pas quel mot il doit classer dans la phrase.
À l'inverse, si $k$ est l'indice du token à classer et si on n'utilise que les $k$ premiers tokens de la séquence, les tokens suivants ne sont pas utilisés.
Pour résoudre ce problème, on va utiliser une couche récurrente **bidirectionnelle**.

Une couche récurrente bidirectionnelle considère une séquence dans les deux sens :
* le sens avant : $ x^{(0)} \longrightarrow x^{(1)} \longrightarrow \ldots \longrightarrow x^{(t-1)} \longrightarrow x^{(t)} $
* le sens arrière : $ x^{(0)} \longleftarrow x^{(1)} \longleftarrow \ldots \longleftarrow x^{(t-1)} \longleftarrow x^{(t)} $

L'image ci-dessous illustre ce principe :

[<img src="../figures/rnn_bidir.png" width="300"/>](../figures/rnn_bidir.png)


### Opération effectuée ici

**Si $k$ est l'indice du token d'intérêt, on va extraire les tenseurs $\overrightarrow{h}^{(k)}$ (l'état caché pour le token d'intérêt dans le sens avant) et $\overleftarrow{h}^{(k)}$ (l'état caché pour le token d'intérêt dans le sens arrière)**.
De cette manière, on indique au modèle quel mot on souhaite classer tout en utilisant tous les mots de la phrase.

L'image ci-dessous illustre l'opération effectuée ici :

[<img src="../figures/brnn_specific_token.png" width="450"/>](../figures/brnn_specific_token.png)


### Quelques mots sur les séquences de longueur variable

Une particularité des données textuelles est que la longueur des séquences n'est pas fixe : le nombre de mots dans une phrase varie d'une phrase à l'autre.
Quand on travaille sur un lot de séquences, on ne peut pas créer directement un nouveau tenseur en dimension supérieure contenant toutes les séquences car les séquences peuvent être de longueur différente.

Pour remédier à ce problème, une solution possible est de faire du **rembourrage** pour les séquences les plus courtes.
Ainsi, les séquences (éventuellement rembourrées) sont toutes de la même longueur et on peut les traiter efficacement en lot.
Cependant, ce n'est qu'une **astuce computationnelle** : on ne souhaite pas utiliser le rembourrage dans le traitement.
Par conséquent, il faut également indiquer quelles valeurs ont été rembourrées pour ne pas les traiter.

PyTorch fournit des outils pour implémenter cette solution :

* [`torch.nn.utils.rnn.pad_sequence()`](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html) : cette fonction rembourre une liste de tenseurs de longueur variable.
* [`torch.nn.utils.rnn.pack_padded_sequence()`](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html) : cette fonction emballe un tenseur contenant des séquences de longueur variable rembourrées.
* [`torch.nn.utils.rnn.pad_packed_sequence()`](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_packed_sequence.html) : cette fonction rembourre un lot emballé de séquences de longueur variable (opération inverse de [`pack_padded_sequence()`](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pack_padded_sequence.html)).

Voici comment utiliser ces outils :

1. Lorsque vous avez affaire à des séquences de longueur variable, l'entrée d'une couche récurrente doit être une instance de [PackedSequence](https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.PackedSequence.html). Pour obtenir une telle instance de cette classe, il faut d'abord rembourrer les séquences avec `pad_sequence()` puis emballer ces séquences avec `pack_padded_sequence()`.
2. Lorsque l'entrée d'une couche récurrente est une instance de `PackedSequence`, sa sortie est également une instance de `PackedSequence`. Il suffit de déballer ces séquences avec `pad_packed_sequence()`.

Voici une illustration de l'utilisation de ces outils :

```python
import torch
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

sequence1 = torch.rand(6, 50)  # une séquence de longueur 6
sequence2 = torch.rand(10, 50)  # une séquence de longueur 10
sequence3 = torch.rand(9, 50)  # une séquence de longueur 9

sequences = [sequence1, sequence2, sequence3]

# On calcule la longueur des séquences
lens = [sequence.size()[0] for sequence in sequences]

# On rembourre cette liste de séquences
sequences_padded = pad_sequence(sequences)

# On emballe ces séquences éventuellement rembourrées
sequences_packed = pack_padded_sequence(sequences_padded, lens, enforce_sorted=False)

# On rembourre les séquences emballées
sequences_unpacked, lens_unpacked = pad_packed_sequence(sequences_packed)
```

### Question 5

On va construire un réseau de neurones récurrent avec l'architecture suivante :

* Première couche : couche GRU bidirectionnelle avec 100 variables cachées.
* Deuxième couche : couche linéaire avec 1 variable en sortie.

Quelques remarques importantes :

* Comme on a un GRU bidirectionnel, l'état caché est un vecteur de taille $200 = 100 \times 2$ car les deux tenseurs sont automatiquement concaténés.
* On ne veut récupérer que l'état caché du token qui nous intéresse (c'est-à-dire du mot à classer).
* Pour votre implémentation de la méthode `step()`, vous supposerez que le lot (l'argument `batch`) est une liste de tenseurs (on verra ensuite comment obtenir ce résultat).

Complétez le code manquant dans les méthodes `__init__()` et `forward()` de la classe `RecurrentNeuralNetwork()`.

In [22]:
import lightning as L
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torchmetrics import Accuracy


class RecurrentNeuralNetwork(L.LightningModule):  # La classe hérite de la classe lightning.LightningModule

    def __init__(self):
        """Constructeur.

        Dans le constructeur, on exécute le constructeur de la clase mère et on définit
        toutes les couches et fonctions d'activation de notre réseau de neurones.
        """
        super().__init__()  # Toujours exécuter le constructeur de la classe mère

        ### BEGIN TODO ###
        self.gru = torch.nn.GRU(input_size=50, hidden_size=100, bidirectional=True)
        self.linear = torch.nn.Linear(in_features=200, out_features=1)
        ### END TODO ###

        self.loss = torch.nn.BCEWithLogitsLoss()
        self.accuracy_train = Accuracy(task="binary")
        self.accuracy_val = Accuracy(task="binary")
        self.accuracy_test = Accuracy(task="binary")

    def forward(self, x, index):
        """Implémente la passe avant.

        L'argument x est une liste de tenseurs correspondant soit à l'entrée une seule
        observation soit aux entrées d'un lot d'observations.
        """
        ### BEGIN TODO ###
        # Calcule la longueur de chaque séquence
        lens = [sequence.size()[0] for sequence in x]

        # Rembourre les séquences les plus petites
        x_padded = pad_sequence(x)

        # Emballe les séquences rembourrées
        x_packed = pack_padded_sequence(x_padded, lens, enforce_sorted=False)

        # Applique la couche GRU
        output_packed, _ = self.gru(x_packed)

        # Déballe la sortie de la couche GRU
        output_unpacked, _ = pad_packed_sequence(output_packed)

        # Récupère l'état caché pour chaque token d'intérêt
        h = torch.concat([output_unpacked[idx, i].reshape(1, -1) for i, idx in enumerate(index)])

        # Applique la couche linéaire
        y = torch.squeeze(self.linear(h))
        # y =
        #### END TODO ####
        return y

    def step(self, batch, dataset):
        """Effectue une étape.

        Une étape consiste à passer d'un lot d'observations (l'argument batch)
        à l'évaluation de la fonction de coût pour ce lot d'observations.

        Parameters
        ----------
        batch : tuple
            Un lot d'observations. Le premier élément du tuple est le lot
            des entrées, le second est le lot des labels.

        dataset : {"training", "validation", "test"}
            Jeu de données utilisé.

        Returns
        -------
        loss : Tensor, shape = (1,)
            La fonction de coût pour ce lot d'observations.
        """
        # Récupère les données du lot d'observations
        X = [item[0].to(self.device) for item in batch]
        y = torch.tensor([item[1] for item in batch], device=self.device)
        index = torch.tensor([item[2] for item in batch], device=self.device)

        logits = self(X, index)  # Passe avant, qui renvoie les logits
        loss = self.loss(logits, y)  # Évaluation de la fonction de coût
        y_pred = (logits > 0).to(torch.float32)  # Prédictions du modèle

        if dataset == "training":
            metric = self.accuracy_train
            name = "train"
            bar_step = True
        elif dataset == "validation":
            metric = self.accuracy_val
            name = "val"
            bar_step = False
        else:
            metric = self.accuracy_test
            name = "test"
            bar_step = False

        acc = metric(y_pred, y)  # Évaluation de la métrique
        self.log(f"loss_{name}", loss, batch_size=len(y), prog_bar=bar_step, on_step=bar_step, on_epoch=True)
        self.log(f"accuracy_{name}", acc, batch_size=len(y), prog_bar=bar_step, on_step=bar_step, on_epoch=True)

        return loss

    def training_step(self, batch):
        """Effectue une étape d'entraînement."""
        return self.step(batch, "training")

    def validation_step(self, batch):
        """Effectue une étape de validation."""
        return self.step(batch, "validation")

    def test_step(self, batch):
        """Effectue une étape d'évaluation."""
        return self.step(batch, "test")

    def on_train_start(self):
        """Code exécuté au début de l'entraînement."""
        string = f"Version {self.trainer.logger.version}"
        print(f"{string}\n{'=' * len(string)}\n")

    def on_train_epoch_end(self):
        """Code exécuté à la fin de chaque époque d'entraînement."""
        metrics = self.trainer.callback_metrics
        string = (f"""
            Époque {self.trainer.current_epoch + 1} / {self.trainer.max_epochs}
            -------------------------------------------------
            |     Jeu      | Fonction de perte | Exactitude |
            | ------------ | ----------------- | ---------- |
            | Entraînement |{metrics['loss_train'].item():^19.5f}|{metrics['accuracy_train'].item():^12.3%}|
            |  Validation  |{metrics['loss_val'].item():^19.5f}|{metrics['accuracy_val'].item():^12.3%}|
            -------------------------------------------------
        """)
        string = '\n'.join([line.strip() for line in string.strip().split('\n')])
        print(string, "\n")

    def configure_optimizers(self):
        """Configure l'algorithme d'optimisation à utiliser."""
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer

Visualisons un résumé de l'architecture de notre modèle.

## Quelques mots concernant le `DataLoader`

On a vu qu'une instance [`torch.utils.data.DataLoader()`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) permet de récupérer des lots d'observations de taille `batch_size`, avec un éventuel mélange des observations à chaque époque (si `shuffle=True`).
Cependant, un `DataLoader` effectue bien plus de choses sous le capot.

En particulier, avec les valeurs par défaut de certains hyperparamètres, il transforme une liste de tenseurs en un tenseur. Par exemple :

* Si chaque entrée est un tenseur à 1 dimension de taille $50$ et si `batch_size=6`, alors la taille du tenseur renvoyé (correspondant au lot d'observations) est de taille $(6, 50)$.
* Si chaque entrée est un tenseur à 2 dimensions de taille $(32, 32)$ et si `batch_size=4`, alors la taille du tenseur renvoyé (correspondant au lot d'observations) est de taille $(4, 32, 32)$.

Le tenseur renvoyé a une dimension supplémentaire (la première dimension) correspondant aux observations.

Comme on l'a vu dans la section précédente, une telle transformation est impossible avec des tenseurs de longueur différente.
Par conséquent, il est nécessaire de désactiver cette transformation et d'implémenter notre propre transformation (avec le rembourrage et l'emballage).

L'hyperparamètre de `DataLoader` qui contrôle cette transformation est `collate_fn` (voir la [documentation](https://pytorch.org/docs/stable/data.html#working-with-collate-fn) concerant cette hyperparamètre).
Sa valeur doit être `None` ou un appelable (*callable*), donc en pratique une fonction.
On peut désactiver la transformation en donnant la fonction identité comme argument pour cet hyperparamètre, soit `collate_fn=lambda x: x`.

En effectuant ce changement, le `DataLoader` renverra maintenant une liste de longueur `batch_size`, chaque élément de cette liste étant ce qui est renvoyé par la méthode `__getitem__()` de l'instance de la classe personnalisée héritant de `Dataset`.

> **Remarque** : Une autre possibilité aurait été d'effectuer les étapes de rembourrage et d'emballage dans la fonction `collate_fn`.

### Question 6

Créez votre propre classe `CustomDataset` héritant de la classe [`torch.utils.data.Dataset()`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset). Pour rappel, cette classe doit définir les trois méthodes suivantes :

* `__len__()` : cette méthode renvoie le nombre d'observations du jeu de données.
* `__getitem__()` : cette méthode charge et renvoie la $n$-ième observation du jeu de données (où $n$ est un argument de la méthode)
* `__init__()` : cette méthode définit toutes les informations nécessaires pour l'implémentation des deux autres méthodes.

Créez ensuite un `Dataloader` pour chacun des trois jeux (entraînement, validation, évaluation) avec des lots de taille $64$ (`batch_size=64`).
N'oubliez pas de mélanger les observations dans le jeu d'entraînement (`shuffle=True`).

In [24]:
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, X, y, index):
        self.X = X
        self.y = y
        self.index = index

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.index[idx]


dataloader_train = DataLoader(
    CustomDataset(X_train, y_train, index_train),
    batch_size=64, shuffle=True, collate_fn=lambda x: x
)

dataloader_val = DataLoader(
    CustomDataset(X_val, y_val, index_val),
    batch_size=64, shuffle=False, collate_fn=lambda x: x
)

dataloader_test = DataLoader(
    CustomDataset(X_test, y_test, index_test),
    batch_size=64, shuffle=False, collate_fn=lambda x: x
)

On va maintenant entraîner notre modèle pendant 10 époques.

In [25]:
from lightning.pytorch.callbacks import TQDMProgressBar
from lightning.pytorch.loggers import CSVLogger


model = RecurrentNeuralNetwork()

trainer = L.Trainer(
    max_epochs=10,
    enable_model_summary=False,  # supprimer le résumé du modèle
    logger=CSVLogger('.'),  # sauvegarder les résultats dans un fichier CSV
    num_sanity_val_steps=0,  # ne pas effectuer d'étape de validation avant l'entraînement
    callbacks=[TQDMProgressBar(refresh_rate=10)]  # mettre à jour la barre de progression tous les 10 lots
)

trainer.fit(
    model=model,
    train_dataloaders=dataloader_train,
    val_dataloaders=dataloader_val
)

Missing logger folder: ./lightning_logs


Training: |                                               | 0/? [00:00<?, ?it/s]

Version 0



Validation: |                                             | 0/? [00:00<?, ?it/s]


Époque 1 / 10
-------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude |
| ------------ | ----------------- | ---------- |
| Entraînement |      0.54904      |  72.283%   |
|  Validation  |      0.56488      |  71.145%   |
-------------------------------------------------



...


Époque 8 / 10
-------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude |
| ------------ | ----------------- | ---------- |
| Entraînement |      0.26580      |  88.828%   |
|  Validation  |      0.52862      |  77.271%   |
-------------------------------------------------



Validation: |                                             | 0/? [00:00<?, ?it/s]


Époque 9 / 10
-------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude |
| ------------ | ----------------- | ---------- |
| Entraînement |      0.22686      |  91.071%   |
|  Validation  |      0.54876      |  76.975%   |
-------------------------------------------------



Validation: |                                             | 0/? [00:00<?, ?it/s]


Époque 10 / 10
-------------------------------------------------
|     Jeu      | Fonction de perte | Exactitude |
| ------------ | ----------------- | ---------- |
| Entraînement |      0.18555      |  92.941%   |
|  Validation  |      0.58629      |  76.806%   |
-------------------------------------------------



## Optimisation des hyperparamètres

La classe codée ci-dessus n'est pas très flexible, dans le sens où si on veut changer les valeurs de certains hyperparamètres alors on doit créer une nouvelle classe.
Il vaut mieux avoir une classe flexible, avec des arguments en entrée permettant de changer les valeurs de ces hyperparamètres à la création d'une instance de cette classe.

La classe `RecurrentNeuralNetworkAdvanced` définie ci-dessous permet de personnaliser un peu l'architecture du modèle.
On a toujours un réseau de neurones récurrent avec une ou plusieurs couches récurrentes suivies d'une ou plusieurs couches linéaires.
Néanmoins, les modifications suivantes sont possibles grâce aux arguments de la classe :

* On peut choisir le type de couche récurrente (RNN / LSTM / GRU).
* On peut choisir le nombre de couches récurrentes.
* On peut choisir le nombre de variables dans l'état caché des couches récurrentes.
* On peut choisir le nombre de couches linéaires ainsi que leurs tailles.
* On peut choisir la fonction d'activation à utiliser entre les couches linéaires.

In [26]:
import lightning as L
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torchmetrics import Accuracy


class RecurrentNeuralNetworkAdvanced(L.LightningModule):
    """Classe générale pour un réseau de neurones récurrent.

    Parameters
    ----------
    rnn_type : {'rnn', 'lstm', 'gru'}
        Type de couche récurrente.

    """

    def __init__(
        self,
        rnn_type='gru',
        n_rnn=1,
        hidden_size_rnn=100,
        n_linear=1,
        output_size_linear=100,
        activation=torch.nn.ReLU()
    ):
        """Constructeur.

        Dans le constructeur, on exécute le constructeur de la clase mère et on définit
        toutes les couches et fonctions d'activation de notre réseau de neurones.
        """
        super().__init__()  # Toujours exécuter le constructeur de la classe mère

        # Couches récurrentes
        if rnn_type == 'rnn':
            self.rnn = torch.nn.RNN(
                50, hidden_size=hidden_size_rnn, num_layers=n_rnn, bidirectional=True
            )
        elif rnn_type == 'lstm':
            self.rnn = torch.nn.LSTM(
                50, hidden_size=hidden_size_rnn, num_layers=n_rnn, bidirectional=True
            )
        elif rnn_type == 'gru':
            self.rnn = torch.nn.GRU(
                50, hidden_size=hidden_size_rnn, num_layers=n_rnn, bidirectional=True
            )
        else:
            raise ValueError("rnn must be one of 'rnn', 'lstm' or 'gru'.")

        # Couches linéaires
        size_linear = [2 * hidden_size_rnn]

        if isinstance(output_size_linear, int):
            size_linear += [output_size_linear] * (n_linear - 1)
        else:
            size_linear += list(output_size_linear)
        size_linear += [1]

        self.linear = torch.nn.Sequential(
            *(
                [torch.nn.Sequential(torch.nn.Linear(in_features, out_features), activation)
                 for in_features, out_features in zip(size_linear[:-2], size_linear[1:-1])]
                + [torch.nn.Linear(size_linear[-2], size_linear[-1])]
            )
        )

        self.loss = torch.nn.BCEWithLogitsLoss()
        self.accuracy_train = Accuracy(task="binary")
        self.accuracy_val = Accuracy(task="binary")
        self.accuracy_test = Accuracy(task="binary")

    def forward(self, x, index):
        """Implémente la passe avant.

        L'argument x est une liste de tenseurs correspondant soit à l'entrée une seule
        observation soit aux entrées d'un lot d'observations.
        """
        # Calcule la longueur de chaque séquence
        lens = [sequence.size()[0] for sequence in x]

        # Rembourre les séquences les plus petites
        x_padded = pad_sequence(x)

        # Emballe les séquences rembourrées
        x_packed = pack_padded_sequence(x_padded, lens, enforce_sorted=False)

        # Applique la couche GRU
        output_packed, _ = self.rnn(x_packed)

        # Déballe la sortie de la couche GRU
        output_unpacked, _ = pad_packed_sequence(output_packed)

        # Récupère l'état caché pour chaque token d'intérêt
        h = torch.concat([output_unpacked[idx, i].reshape(1, -1) for i, idx in enumerate(index)])

        # Applique la couche linéaire
        y = torch.squeeze(self.linear(h))

        return y

    def step(self, batch, dataset):
        """Effectue une étape.

        Une étape consiste à passer d'un lot d'observations (l'argument batch)
        à l'évaluation de la fonction de coût pour ce lot d'observations.

        Parameters
        ----------
        batch : tuple
            Un lot d'observations. Le premier élément du tuple est le lot
            des entrées, le second est le lot des labels.

        dataset : {"training", "validation", "test"}
            Jeu de données utilisé.

        Returns
        -------
        loss : Tensor, shape = (1,)
            La fonction de coût pour ce lot d'observations.
        """
        # Récupère les données du lot d'observations
        X = [item[0].to(self.device) for item in batch]
        y = torch.tensor([item[1] for item in batch], device=self.device)
        index = torch.tensor([item[2] for item in batch], device=self.device)

        logits = self(X, index)  # Passe avant, qui renvoie les logits
        loss = self.loss(logits, y)  # Évaluation de la fonction de coût
        y_pred = (logits > 0).to(torch.float32)  # Prédictions du modèle

        if dataset == "training":
            metric = self.accuracy_train
            name = "train"
            bar_step = True
        elif dataset == "validation":
            metric = self.accuracy_val
            name = "val"
            bar_step = False
        else:
            metric = self.accuracy_test
            name = "test"
            bar_step = False

        acc = metric(y_pred, y)  # Évaluation de la métrique
        self.log(f"loss_{name}", loss, batch_size=len(y), prog_bar=bar_step, on_step=bar_step, on_epoch=True)
        self.log(f"accuracy_{name}", acc, batch_size=len(y), prog_bar=bar_step, on_step=bar_step, on_epoch=True)

        return loss

    def training_step(self, batch):
        """Effectue une étape d'entraînement."""
        return self.step(batch, "training")

    def validation_step(self, batch):
        """Effectue une étape de validation."""
        return self.step(batch, "validation")

    def test_step(self, batch):
        """Effectue une étape d'évaluation."""
        return self.step(batch, "test")

    def on_train_start(self):
        """Code exécuté au début de l'entraînement."""
        string = f"Version {self.trainer.logger.version}"
        print(f"{string}\n{'=' * len(string)}\n")

    def on_train_epoch_end(self):
        """Code exécuté à la fin de chaque époque d'entraînement."""
        metrics = self.trainer.callback_metrics
        string = (f"""
            Époque {self.trainer.current_epoch + 1} / {self.trainer.max_epochs}
            ------------------------------------------------
            |     Jeu      | Fonction de coût | Exactitude |
            | ------------ | ---------------- | ---------- |
            | Entraînement |{metrics['loss_train'].item():^18.6f}|{metrics['accuracy_train'].item():^12.3%}|
            |  Validation  |{metrics['loss_val'].item():^18.6f}|{metrics['accuracy_val'].item():^12.3%}|
            ------------------------------------------------
        """)
        string = '\n'.join([line.strip() for line in string.strip().split('\n')])
        print(string, "\n")

    def configure_optimizers(self):
        """Configure l'algorithme d'optimisation à utiliser."""
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer

### Question 7

Initialisez quelques modèles avec des combinaisons différentes pour les valeurs de ces hyperparamètres.
N'entraînez pas les modèles, mais effectuez une passe avant sur un petit lot d'observations pour vous assurer qu'aucune erreur n'est levée.

In [27]:
RecurrentNeuralNetworkAdvanced(
    rnn_type='gru', n_rnn=1, hidden_size_rnn=100, n_linear=1, output_size_linear=100
)(X_train[:3], index_train[:3])

tensor([-0.0928, -0.0776, -0.2448], grad_fn=<SqueezeBackward0>)

In [28]:
RecurrentNeuralNetworkAdvanced(
    rnn_type='lstm', n_rnn=2, hidden_size_rnn=200, n_linear=3,
    output_size_linear=(200, 100), activation=torch.nn.LeakyReLU()
)(X_train[:3], index_train[:3])

tensor([-0.0976, -0.0991, -0.0989], grad_fn=<SqueezeBackward0>)

In [29]:
RecurrentNeuralNetworkAdvanced(
    rnn_type='rnn', n_rnn=3, hidden_size_rnn=30, n_linear=4, output_size_linear=(200, 120, 50),
)(X_train[:3], index_train[:3])

tensor([-0.0249, -0.0246, -0.0146], grad_fn=<SqueezeBackward0>)

### Question 8

Utilisez votre nouvelle classe personnalisée pour initialiser un modèle avec une architecture différente du premier modèle.
**N'oubliez pas que plus votre modèle est profond et a de paramètres entraînables, plus l'entraînement prendra du temps.**
Si vous avez assez de temps, essayez plusieurs architectures.
Dans tous les cas, sauvegardez tous vos modèles !

In [30]:
model_2 = RecurrentNeuralNetworkAdvanced(
    rnn_type='lstm', n_rnn=2, hidden_size_rnn=200, n_linear=3,
    output_size_linear=(200, 100), activation=torch.nn.LeakyReLU()
)

trainer_2 = L.Trainer(
    max_epochs=10,
    enable_model_summary=False,  # supprimer le résumé du modèle
    logger=CSVLogger('.'),  # sauvegarder les résultats dans un fichier CSV
    num_sanity_val_steps=0,  # ne pas effectuer d'étape de validation avant l'entraînement
    callbacks=[TQDMProgressBar(refresh_rate=10)]  # mettre à jour la barre de progression tous les 10 lots
)

trainer_2.fit(
    model=model_2,
    train_dataloaders=dataloader_train,
    val_dataloaders=dataloader_val
)

Training: |                                               | 0/? [00:00<?, ?it/s]

Version 1



Validation: |                                             | 0/? [00:00<?, ?it/s]


Époque 1 / 10
------------------------------------------------
|     Jeu      | Fonction de coût | Exactitude |
| ------------ | ---------------- | ---------- |
| Entraînement |     0.536228     |  72.305%   |
|  Validation  |     0.559583     |  70.131%   |
------------------------------------------------



Validation: |                                             | 0/? [00:00<?, ?it/s]

...


Époque 8 / 10
------------------------------------------------
|     Jeu      | Fonction de coût | Exactitude |
| ------------ | ---------------- | ---------- |
| Entraînement |     0.159763     |  93.370%   |
|  Validation  |     0.677258     |  78.918%   |
------------------------------------------------



Validation: |                                             | 0/? [00:00<?, ?it/s]


Époque 9 / 10
------------------------------------------------
|     Jeu      | Fonction de coût | Exactitude |
| ------------ | ---------------- | ---------- |
| Entraînement |     0.123253     |  95.124%   |
|  Validation  |     0.774289     |  79.172%   |
------------------------------------------------



Validation: |                                             | 0/? [00:00<?, ?it/s]


Époque 10 / 10
------------------------------------------------
|     Jeu      | Fonction de coût | Exactitude |
| ------------ | ---------------- | ---------- |
| Entraînement |     0.092685     |  96.359%   |
|  Validation  |     0.881185     |  79.383%   |
------------------------------------------------



### Question 9

En théorie, il faudrait choisir le meilleur modèle sur le jeu de validation et l'évaluer sur le jeu d'évaluation. Par curiosité, on va ici évaluer tous les modèles sur le jeu d'évaluation.

In [31]:
trainer.validate(model, dataloader_val)

Validation: |                                             | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       accuracy_val        │    0.7680608630180359     │
│         loss_val          │    0.5862944722175598     │
└───────────────────────────┴───────────────────────────┘

[{'loss_val': 0.5862944722175598, 'accuracy_val': 0.7680608630180359}]

In [32]:
trainer_2.validate(model_2, dataloader_val)

Validation: |                                             | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       accuracy_val        │    0.7938318252563477     │
│         loss_val          │     0.881184995174408     │
└───────────────────────────┴───────────────────────────┘

[{'loss_val': 0.881184995174408, 'accuracy_val': 0.7938318252563477}]

In [33]:
trainer.test(model, dataloader_test)

Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       accuracy_test       │    0.7844929695129395     │
│         loss_test         │    0.5439191460609436     │
└───────────────────────────┴───────────────────────────┘

[{'loss_test': 0.5439191460609436, 'accuracy_test': 0.7844929695129395}]

In [34]:
trainer_2.test(model_2, dataloader_test)

Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       accuracy_test       │    0.8019298911094666     │
│         loss_test         │     0.874323844909668     │
└───────────────────────────┴───────────────────────────┘

[{'loss_test': 0.874323844909668, 'accuracy_test': 0.8019298911094666}]